# Project 2: Multi-location Transshipment Problem 

In this project, we compare Stochastic Gradient Descent Vs. Bender's Decomposition in optimizing a Two-Stage Stochastic LP.

In [1]:
!pip install autotime
!pip install nb-black
!pip install cvxpy

  Using cached autotime-0.1.5-py3-none-any.whl (46 kB)
  Using cached dnntime-0.4.1-py3-none-any.whl (47 kB)
  Using cached fbprophet-0.7.1.tar.gz (64 kB)
  Using cached pyflux-0.4.15.tar.gz (1.3 MB)


  ERROR: Command errored out with exit status 1:
   command: /opt/anaconda3/envs/jeff/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-cyipebue/fbprophet/setup.py'"'"'; __file__='"'"'/tmp/pip-install-cyipebue/fbprophet/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-46ektaws
       cwd: /tmp/pip-install-cyipebue/fbprophet/
  Complete output (38 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib
  creating build/lib/fbprophet
  creating build/lib/fbprophet/stan_model
  Traceback (most recent call last):
    File "<string>", line 1, in <module>
    File "/tmp/pip-install-cyipebue/fbprophet/setup.py", line 149, in <module>
      long_description_content_type='text/markdown',
    File "/opt/anaconda3/envs/jeff/lib/python3.7/site-packages/setuptoo

  ERROR: Command errored out with exit status 1:
   command: /opt/anaconda3/envs/jeff/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-cyipebue/pyflux/setup.py'"'"'; __file__='"'"'/tmp/pip-install-cyipebue/pyflux/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-qzqjec6v
       cwd: /tmp/pip-install-cyipebue/pyflux/
  Complete output (628 lines):
  running bdist_wheel
  running build
  running config_cc
  unifing config_cc, config, build_clib, build_ext, build commands --compiler options
  running config_fc
  unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
  running build_src
  build_src
  building extension "pyflux.__check_build._check_build" sources
  building extension "pyflux.arma.arma_recursions" sources
  building extension "pyflux.arma.nn_architecture"

Failed to build fbprophet pyflux
    Running setup.py install for fbprophet ... error
    ERROR: Command errored out with exit status 1:
     command: /opt/anaconda3/envs/jeff/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-cyipebue/fbprophet/setup.py'"'"'; __file__='"'"'/tmp/pip-install-cyipebue/fbprophet/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-7vad5ilk/install-record.txt --single-version-externally-managed --compile --install-headers /opt/anaconda3/envs/jeff/include/python3.7m/fbprophet
         cwd: /tmp/pip-install-cyipebue/fbprophet/
    Complete output (40 lines):
    running install
    running build
    running build_py
    creating build
    creating build/lib
    creating build/lib/fbprophet
    creating build/lib/fbprophet/stan_model
    Traceback (most recent call last)

You should consider upgrading via the '/opt/anaconda3/envs/jeff/bin/python -m pip install --upgrade pip' command.


In [1]:
# %load_ext autotime
# %load_ext nb_black

# General
import time
import numpy as np
import cvxpy as cp
import scipy as sp
import pandas as pd
import itertools

# Ignore warnings LOL
import warnings

warnings.simplefilter("ignore")

# SEED FOR RANDOMNESS
np.random.seed(420)

---
## 1. Preface

The type of problem we care about in this project are known as [two-stage stochastic programming](https://en.wikipedia.org/wiki/Stochastic_programming#Two-stage_problems). Let The general formulation of it is as follows (Adapted from Wikipedia):

### 1.1. General Two-Stage Stochastic Program

#### 1.1.1 First Stage Stochastic Program

\begin{align}
    \min_{x\in X} & \{ g(x)= f(x) + E_{\tilde{s}\sim S}[h(x, s=\tilde{s})]\}
\end{align}

, where $h(x, s=\tilde{s})$ is the optimal value of the second-stage problem given a specific scenario $\tilde{s}$. $x \in X$ is our first-stage decision variable vector, $S$ is a probability distribution of the scenario variables that we're sampling from.

#### 1.1.2 Second Stage Stochastic Program

For a specific $\tilde{x}$ and $\tilde{s}$,

\begin{align}
    h(x=\tilde{x}, s=\tilde{s}) = \min_{y} & \{ d(s, y) \,\vert\,C(s, x) +D(s, y) = \xi(s)\}  \\
\end{align}

, where $C, D, \xi$ are functions given a specific scenario $\tilde{s}$. $y$ is our second-stage decision variable vector. 

At the first stage we optimize (minimize in the above formulation) the cost of the first-stage decision plus the expected cost of the (optimal) second-stage decision. We can view the second-stage problem simply as an optimization problem which describes our supposedly optimal behavior when the uncertain data is revealed (specific scenario $\tilde{s}$), or we can consider its solution as a recourse action where the term $D(s, y)$ compensates for a possible inconsistency of the system $C(s, x) \leq \xi(s)$ and $d(s, y)$ is the cost of this recourse action.

### 1.2. Two-Stage Stochastic Linear Program

The stochastic linear program below is just a specific instance of the general program above, with very minor changes - e.g. functions of first-stage / second-stage decision variables are no longer general functions, but rather vectors / matrices because of constraints must be linear for a linear program.

#### 1.2.1 First Stage Stochastic Linear Program

\begin{align}
    \min\limits_{x\in \mathbb{R}^n} & f(x)= c^\top x + E_{\tilde{s}\sim S}[h(x, s=\tilde{s})] & \\
    \text{subject to} 
    & Ax = b &\\
    & x \geq 0
\end{align}

, where $h(x, s=\tilde{s})$ is the optimal value of the second-stage problem given a specific scenario $\tilde{s}$. $x \in \mathbb{R}^n$ is our first-stage decision variable vector, $S$ is a probability distribution of the scenario variables that we're sampling from., where $h(x, s)$ is the optimal value of the second-stage problem

#### 1.2.2 Second Stage Stochastic Linear Program

For a specific $\tilde{x}$ and $\tilde{s}$,

\begin{align}
    h(x=\tilde{x}, s=\tilde{s}) = \min\limits_{y\in \mathbb{R}^m}\,& d(s)^T y \\
    \text{subject to }\,& C(s)x + D(s)y = \xi(s) \\
    &y \geq 0
\end{align}

, where $C, D, \xi$ are matrices given a specific scenario $\tilde{s}$. $y$ is our second-stage decision variable vector. 

At the first stage we optimize (minimize in the above formulation) the cost of the first-stage decision plus the expected cost of the (optimal) second-stage decision. We can view the second-stage problem simply as an optimization problem which describes our supposedly optimal behavior when the uncertain data is revealed (specific scenario $\tilde{s}$), or we can consider its solution as a recourse action where the term $D(s)y$ compensates for a possible inconsistency of the system $C(s)x \leq \xi(s)$ and $d(s)^\top y$ is the cost of this recourse action.

### 1.3. Two-Stage Stochastic Linear Program with Discretized Scenario Distribution

What happens if we can't get / don't want to sample from a continuous distribution for the scenarios? (What if $S$ is discrete instead of continuous?). Nothing much changes other than how we calculate the expectation of the second-stage objective values. 

#### 1.3.1 First Stage Stochastic Linear Program with Discretized Scenario Distribution

\begin{align}
    \min\limits_{x\in \mathbb{R}^n}\,& f(x)= c^\top x + \underbrace{\sum_{s = \tilde{s}} p(s = \tilde{s}) h(x, s = \tilde{s})}_{E_{\tilde{s}\sim S}[h(x, s=\tilde{s})]} \\
    \text{subject to }\,
    & Ax = b \\
    & x \geq 0
\end{align}

, where $h(x, s)$ is the optimal value of the second-stage problem, and $p(s = \tilde{s})$ is the probability of the scenario $\tilde{s}$ occuring.

#### 1.3.2 Second Stage Stochastic Linear Program with Discretized Scenario Distribution

For a specific $\tilde{x}$ and $\tilde{s}$,

\begin{align}
    h(x=\tilde{x}, s=\tilde{s}) = \min\limits_{y\in \mathbb{R}^m}\,& d(s)^T y \\
    \text{subject to} & \, C(s)x + D(s)y = \xi(s) \text{, where }C(s), D(s), \xi(s) \\
    & y \geq 0 \\
\end{align}

However, how do we go about solving this two-stage stochastic programs? The problem we face here is We will present two algorithms: Stochastic Gradient Descent and L-Shaped / Bender's Method.

---
## 2. Method: Stochastic Gradient Descent

To keep things simple, we will work on using stochastic gradient descent on a stochastic linear program with s discretized scenario distribution.

First, we recall the general key update formula in gradient descent (adjusting parameter $w_i$ in negative direction of gradient of objective $f$):

\begin{align}
    & w_i \leftarrow w_i + \alpha \frac{\partial f}{\partial w_i}\,\forall i = 0, \cdots , \text{Number of parameters} \\
\end{align}

Let's recall the Lagrangian for the first stage problem :

\begin{align}
    \mathcal{L}(x) = c^\top x + E_{\tilde{s}\sim S}[h(x, s=\tilde{s})] - \mu
\end{align}

Assume we have solved the second stage problem, and have retrieved optimal primal variables $y^*$ and dual variables $\pi^*$ (equality constraints) and $\lambda^*$ (inequality constraints), then recall the Lagrangian of $h(x=\tilde{x}, s=\tilde{s})$:

\begin{align}
  \mathcal{L}(x) &= d(s)^\top y^* - \pi^* \left(C(s)x + D(s)y^* - \xi(s) \right) - \lambda^* y^* \\
\end{align}

To perform gradient descent on our obejctive function $f(x)$, we need to find its gradient / subgradient w.r.t. $x$:

\begin{align}
  f(x) &= c^\top x + \sum_s p_s h_s(x) \\
  \frac{df}{dx} &= c + \frac{d \left(\sum_s p_s h_s(x)\right)}{dx} \\
  &= c + \sum_s p_s\frac{d\left(h_s(x)\right)}{dx} \\
  &= c + \sum_s p_s\frac{d\left(\mathcal{L}(x)\right)}{dx} \\
  &= c + \sum_s p_s\frac{d\left(d^\top y_s - \pi_s \left( Dy_s - \xi_s + C_s x \right) - \lambda_s y_s \right)}{dx} \\
  &= c - \sum_s p_s (\pi^\top_s C_s) \\
\end{align}

Stochastic Gradient Descent:

\begin{align}
  x &= x + \alpha \frac{df}{dx} \\
  &= x + \alpha \left( c - \sum_s p_s (\pi^\top_s C_s) \right) \\
\end{align}


Orthogonal Projection onto [intersection of half-spaces (Convex Polytope)](https://en.wikipedia.org/wiki/Convex_polytope)

\begin{align}
    \arg \min_{x \geq 0} & \frac{1}{2} {\lVert x - y \rVert}_{2}^{2} \\
    \text{subject to } & Ax \leq b
\end{align}

Lagrangian

\begin{align}
    \mathcal{L}(x, \lambda) &= \frac{1}{2} {\lVert x - y \rVert}_{2}^{2} + \lambda \left(Ax  - b \right) \\
\end{align}

Lagrange Dual

\begin{align}
    \mathcal{L}(\lambda) 
    &= \inf_{x \geq 0} \mathcal{L}(x, \lambda) \\ 
    &= \inf_{x \geq 0} \frac{1}{2} {\lVert x - y \rVert}_{2}^{2} + \lambda \left(Ax  - b \right) \\
\end{align}

KKT Conditions

Stationarity:

\begin{align}
    \nabla_x \mathcal{L}(x, \lambda) &= 0 \\
    \nabla_x \frac{1}{2} {\left( x - y \right) }^\top \left( x - y \right) + \lambda \left(Ax  - b \right) &= 0 \\
    \left( x - y \right) + \lambda A &= 0 \\
    \lambda &= A^{-1}\left(y - x\right) \\
\end{align}



In [3]:
class StochasticGradientDescent:
    @staticmethod
    def projection(vector, constraints=None):
        """Find the projection of y onto the C(A) that minimizes
        the euclidean norm between y and the projected vector
        
        .. math::
            \begin{align}
                \arg \min_{x \geq 0} & \frac{1}{2} {\lVert x - y \rVert}_{2}^{2} \\
                \text{subject to } & x \in X, \, \text{where }X\text{ is a convex set}
            \end{align}
            
        Args:
            vector (np.array): Vector to be projected
            
        Returns:
            np.array: Projected vector
        """
        P_positive = lambda vector: np.maximum(
            vector, np.zeros(vector.shape)
        )  # Positive projection operator on a point
        if constraints is None:
            return P_positive(
                vector
            )  # Return positive projection if there isnt a convex polytope / set constraint

        proj = cp.Variable(vector.shape, nonneg=True)  # Projected Vector
        obj = cp.Minimize(
            cp.norm2(proj - vector)
        )  # Minimize euclidean norm of difference between y and projection
        prob = cp.Problem(obj, constraints=constraints)
        prob.solve()
        return proj.value

    @staticmethod
    def projection_dykstra(projections, x0, max_iter=1000, tol=1e-6):
        """Dykstra's Projection algorithm to find the orthogonal
        projection of a point onto a convex polytope (intersection
        of affine halfspaces). The algorithm below is adapted from
        https://github.com/mjhough/Dykstra/blob/main/dykstra/Dykstra.py

        Args:
            projections

        """
        assert len(x0.shape) == 1, "x0 must be a vector"

        x = x0.copy()
        y = np.zeros((projections.shape[0], x0.shape[0]))

        n = 0
        cI = np.inf
        while n < max_iter and cI >= tol:
            cI = 0
            for i in range(projections.shape[0]):

                # Update iterate
                prev_x = x.copy()
                x = projections[i](prev_x - y[i, :])

                # Update increment
                prev_y = y[i, :].copy()
                y[i, :] = x - (prev_x - prev_y)

                # Stop condition
                cI += np.linalg.norm(prev_y - y[i, :]) ** 2

                n += 1
        return x

    def __init__(
        self, c, x0, first_stage_constraints, s_gen, n, n_iter=None, α=1e0, ε=1e-04, log=True
    ):
        """The Stochastic Gradient Descent method implemented here
        is used to optimize the Two-Stage Stochastic Linear Program
        with second-stage variables following a discrete distribution

        """
        self.c = c  # First-stage decision coefficient
        self.x0 = x0  # Initial first-stage decision variable values
        self.first_stage_constraints = first_stage_constraints  # Function that gives a list of the constraints for first-stage problem
        self.s_gen = s_gen  # Function to generate scenario specific entities
        self.n = n  # Number of scenarios to generate using s_gen
        self.n_iter = n_iter  # Number of iterations of sgd
        self.α = α  # Learning rate for gradient descent
        self.ε = ε # Tolerance for how close subgradient is to 0
        self.log = log  # Whether or not to log progress

    def solve(self):
        """Solves the Two-stage Stochastic Optimization"""
        f_primal_optimal, x_star, k = self.first_stage()

        if self.log:
            print(
                f"Optimal primal objective of first stage problem: {np.round(f_primal_optimal, 3)}"
            )
            print(f"Optimal first-stage decision variables: {np.round(x_star, 3)}")
            print(f"Number of iterations for convergence: {k}")

        return f_primal_optimal, x_star, k

    def first_stage(self):
        """First-stage Optimization"""
        f_primal_optimal = np.inf  # First-stage primal optimal objective value
        x = (
            np.array([0.0] * self.c.shape[0]) if self.x0 is None else self.x0.copy()
        )  # First-stage decision variables

#         # Fix number of iterations
#         for k in range(self.n_iter):
        k = 0
        while True:
            
            data = list(self.s_gen(self.n))  # Random Scenarios and associated variables

            # Initialize subgradient of f
            ν = self.c.copy()

            # Initialize expectation of h
            expected_h = 0

            # Accumulating subgradients
            for p_s, d_s, C_s, D_s, ξ_s in data:

                π_s, h_primal_optimal = self.second_stage(
                    x=x, d=d_s, C=C_s, D=D_s, ξ=ξ_s
                )

                ν -= p_s * π_s.T @ C_s  # Update subgradient
                expected_h += (
                    p_s * h_primal_optimal
                )  # Update expected second-stage optimal value
                
            # Early stopping condition
            if np.allclose(ν, np.zeros(self.c.shape[0]), atol=self.ε):
                print("Early Stopping condition satisfied.")
                break
            
            # The current f primal optimal found
            current_f_primal_optimal = self.c.T @ x + expected_h

            # Updated x value [WHY TF is it a + and not a -]
            x = StochasticGradientDescent.projection(
                vector=x + (self.α * ν), constraints=self.first_stage_constraints
            )

            if self.log and k % 50 == 0:
                print("=" * 20 + f" Iteration {k}" + "=" * 20)
                print(
                    f"Current Primal Objective Found:",
                    np.round(current_f_primal_optimal, 3),
                    "|",
                    f"Best Primal Objective Found:",
                    np.round(f_primal_optimal, 3),
                )

            # If minimized value is better, update
            if f_primal_optimal > current_f_primal_optimal:
                f_primal_optimal = current_f_primal_optimal
                
            # Update k
            k += 1

        return f_primal_optimal, x, k

    def second_stage(self, x, d, C, D, ξ):
        """Second-stage Optimization"""
        y = cp.Variable(d.shape[0], nonneg=True)  # Second-stage decision variables
        constraints = [C @ x + D @ y == ξ]  # Second-stage constraints
        obj = cp.Minimize(d @ y)
        prob = cp.Problem(obj, constraints=constraints)
        prob.solve()

        return constraints[0].dual_value, prob.value

---
## 3. L-Shaped / Bender's Method

Master Problem:

\begin{align}
  \underset{S \geq 0}{\min} \mu & \\
\end{align}

Sub Problem:

\begin{align}
  \underset{F}{\min} h(S=S^* = \underset{S \geq 0}{\arg\min}\,\mu, d=\tilde{d}) &= \underset{F}{\min} \sum^N_{i=1} h_i F_{B_i E_i} + \sum^N_{i=1} \sum^N_{j=1} c_{ij} F_{B_i M_j} + \sum^N_i p_i F_{RM_i} \\
  \text{subject to } &S_i = F_{B_i M_i} + \sum^N_{j =1 \\ i \not= 1} F_{B_i M_j} + F_{B_i E_i}\qquad i = 1, \cdots , N, \\
  &F_{B_i M_i} + \sum^N_{j = 1 \\ j \not= i} F_{B_j M_i} + F_{RM_i} = {d}_i\qquad i = 1, \cdots , N, \\
  &\sum^N_{i=1} {d}_i = \sum^N_{i=1} F_{RM_i} + \sum^N_{i = 1} F_{RE_i}, \\
  &F_{B_i E_i} + F_{RE_i} = S_i\qquad i = 1, \cdots , N, \\
  &F_{B_i E_i}, F_{B_i M_j}, F_{RM_i}, F_{RE_i} \geq 0\qquad i = 1, \cdots , N, \\
  &j = 1, \cdots , N.
\end{align}

Sp:

\begin{align}
  h_s(x) &= \underset{y_s}{\min} d^\top y_s \\
  \text{subject to } Dy_s &= \xi_s - C_s x \\
  y_s &\geq 0 \\
\end{align}

Dual Sp:

\begin{align}
  \underset{\pi_s, \lambda_s}{\max} & \left[\xi_s - C_s x\right]^\top \left(\pi_s, \lambda_s\right) \\
  \text{subject to } & \pi_s \text{ free} \\
  & \lambda_s \geq 0 \\
\end{align}

In [5]:
class Benders:
    def __init__(self, c, first_stage_constraints, s_gen, ε=1e-5, log=True):
        """The Bender's method implemented here
        is used to optimize the Two-Stage Stochastic Linear Program
        with second-stage variables following a discrete distribution

        """
        self.c = c  # First-stage decision coefficient
        self.first_stage_constraints = first_stage_constraints  # Function that gives a list of the constraints for first-stage problem
        self.s_gen = s_gen  # Function to generate scenario specific entities
        self.ε = ε  # Tolerance between Upper Bound and Lower Bound
        self.log = log  # Whether or not to log progress

    def solve(self):
        """Solves the Two-stage Stochastic Optimization"""
        # 1. Initialize UB = infinity, LB = -infinity
        UB, LB = np.inf, -np.inf
        x = cp.Variable(self.c.shape, nonneg=True)  # First-stage decision variables

        # Master Problem
        z = cp.Variable(1, nonneg=True)
        MP_obj = cp.Minimize(z)

        # Bender cuts that we will be adding to the master problem
        cuts = (
            self.first_stage_constraints
            if self.first_stage_constraints is not None
            else []
        )

        k = 0

        # 2. While UB - LB > ε, we will continue adding cuts
        while UB - LB > self.ε:

            expected_h = 0
            α, β = 0, 0  # Hyperplane of cut coefficients

            # Go through all the scenarios possible
            for p_s, d_s, C_s, D_s, ξ_s in self.s_gen():

                # 3a. Solve the Dual of the Sub-problem for all combinations of demand \tilde{d}
                # [Equivalent to solving primal since LPs have 0 duality gap]
                π_optimal, h_dual_optimal, status = self.second_stage(
                    x=x, d=d_s, C=C_s, D=D_s, ξ=ξ_s
                )

                #                 # 4a. If dual variables are unbounded, add cut
                #                 if status == "infeasible":
                #                     cuts += [(ξ_s - C_s @ x).T @ π_optimal <= 0]

                #                 # 4b. Else set UB and add cut on first stage primal optimal value
                #                 else:
                #                     expected_h += p_s * h_dual_optimal
                #                     cuts += [z >= (ξ_s - C_s @ x).T @ π_optimal]

                expected_h += p_s * h_dual_optimal
                α += p_s * ξ_s.T @ π_optimal
                β += -p_s * C_s.T @ π_optimal

            # 3b. Add a new halfspace constraint 
            cuts += [z >= α + β.T @ x]

            # 4. Update UB
            UB = min(
                UB,
                self.c.T @ x.value + expected_h if x.value is not None else expected_h,
            )

            # 5. Solve Master Problem
            MP_prob = cp.Problem(MP_obj, constraints=cuts)
            MP_prob.solve()

            # 6. Update LB
            LB = MP_prob.value

            if self.log:
                print("=" * 20 + f" Iteration {k}" + "=" * 20)
                print(
                    f"Lower Bound:",
                    np.round(LB, 3),
                    "|",
                    f"Upper Bound:",
                    np.round(UB, 3),
                )
            k += 1

        if self.log:
            print(f"Optimal primal objective of first stage problem: {np.round(LB, 3)}")
            print(f"Optimal first-stage decision variables: {np.round(x.value, 3)}")
            print(f"Number of iterations for convergence: {k}")

        return LB, np.array(x.value), k

    def second_stage(self, x, d, C, D, ξ):
        """Dual Form of Second-stage Optimization"""

        π = cp.Variable(
            ξ.shape, nonneg=False
        )  # Second-stage dual variables are Free since they are dual of the equality constraints
        constraints = [D.T @ π <= d]  # Second-stage dual constraints
        obj = cp.Maximize(
            (ξ - C @ (np.zeros(C.shape[1]) if x.value is None else x.value)).T @ π
        )
        prob = cp.Problem(obj, constraints=constraints)
        prob.solve()

        return π.value, prob.value, prob.status

---
## 4. Two-stage Multi-location Transshipment Problem

### 4.1. Problem Setup

#### 4.1.1 First Stage Stochastic Linear Program

In the first stage problem, we want to figure out what the optimal order-up-to quantity $S$ is. We formulate the first stage LP as follows, minimizing the expected (stochastic demand) transshipment cost with the order-up-to quantities $S$ as our primal variables:

\begin{align}
\underset{S \geq 0}{\min}\,& \mathbb{E}_{\tilde{d} \sim D}\left[h(S, d = \tilde{d})\right] \qquad \text{, where }D\text{ is the demand distribution} \\
\end{align}

#### 4.1.2 Second Stage Stochastic Linear Program

In our second stage problem, given a specific order-up-to quantity $\tilde{S}$, and a fixed demand $\tilde{d}$, we want to solve the following constrained linear program over the primal variables $F$ (flow to and from each node), minimziing the transshipment cost for the specific instance of $\tilde{S}$ and $\tilde{d}$:

\begin{align}
    h(S = \tilde{S}, d=\tilde{d}) = \underset{F}{\min} &\sum^N_{i=1} h_i F_{B_i E_i} + \sum^N_{i=1} \sum^N_{j=1} c_{ij} F_{B_i M_j} + \sum^N_i p_i F_{RM_i} \\
  \text{subject to } & S_i = F_{B_i M_i} + \sum^N_{j =1 \\ i \not= 1} F_{B_i M_j} + F_{B_i E_i} \,\,\,\, i = 1, \cdots , N, \\
  &F_{B_i M_i} + \sum^N_{j = 1 \\ j \not= i} F_{B_j M_i} + F_{RM_i} = {d}_i \,\,\,\, i = 1, \cdots , N, \\
  &\sum^N_{i=1} {d}_i = \sum^N_{i=1} F_{RM_i} + \sum^N_{i = 1} F_{RE_i}, \\
  &F_{B_i E_i} + F_{RE_i} = S_i \,\,\,\, i = 1, \cdots , N, \\
  &F_{B_i E_i}, F_{B_i M_j}, F_{RM_i}, F_{RE_i} \geq 0 \,\,\,\, i = 1, \cdots , N, \\
  &j = 1, \cdots , N.
\end{align}

Reformulate objective and constraints into standard form:

In [6]:
def get_transshipment_variables():

    # Number of retailers
    N = 4

    # Holding cost Vector
    h = np.array([1] * N)

    # Shortage cost Vector
    p = np.array([4] * N)

    # Cost of transshipment from retailer i to retailer j Matrix
    C = np.full((N, N), 0.5)
    np.fill_diagonal(C, 0)

    # PMF of Retailer Demand distribution
    P = np.array([0.020, 0.14, 0.68, 0.14, 0.020])

    # Each Retailer's Demand distribution associated with probabilities above
    RETAILER_DEMANDS = np.array(
        [
            [60, 80, 100, 120, 140],
            [100, 150, 200, 250, 300],
            [90, 120, 150, 180, 210],
            [70, 120, 170, 220, 270],
        ]
    )

    return N, h, p, C, P, RETAILER_DEMANDS


def random_demand(P, retailer_demands):
    """Samples a random demand vector according to demand pmf
    parameters given

    Args:
        P (numpy.array): Array of probabilities for each demand, shape: (Number of probabilities)
        retailer_demands (numpy.array): Matrix of retailer demands, shape: (Number of retailers, Number of discrete demands)

    Returns:
        Tuple[np.array, float]: A tuple of the (sampled Demand Vector, probability of the Sampled Demand Vector)
    """
    assert np.isclose(
        np.sum(P), 1
    ), "p must sum to 1 to be a valid probability distribution."
    sampled_demand = np.array(
        [np.random.choice(retailer_demand, p=P) for retailer_demand in retailer_demands]
    )
    p_s = np.prod(
        [
            P[np.argwhere(retailer_demand == D_i)[0][0]]
            for retailer_demand, D_i in zip(retailer_demands, sampled_demand)
        ]
    )
    return sampled_demand, p_s


def get_second_stage_variables(s):
    """"""

    N, h, p, C, P, RETAILER_DEMANDS = get_transshipment_variables()

    # Coefficient of Second-stage decision variables
    d_s = np.concatenate(
        (
            h,  # Coefficient of Amount of inventory that moves from begining to ending (FBE)
            C.flatten(),  # Coefficient of Amount of inventory to ship from current retailer to other retailers (FBM)
            p,  # Coefficient of Amount to ship from supplier to retailer's middle inventory (FRM)
            np.array(
                [0] * N
            ),  # Coefficient of Amount to ship from supplier to retailer's ending inventory (FRE)
        )
    )

    # Second-stage C(s) matrix
    C_s = np.concatenate(
        (
            np.eye(N),
            np.zeros(shape=(N, N)),
            np.zeros(shape=(1, N)),
            np.eye(N),
        ),
        axis=0,
    )

    # Second-stage D(s) matrix
    D_s = np.concatenate(
        (
            np.concatenate(
                (
                    -np.eye(N),
                    np.array(
                        [
                            ([-1] * N + [0] * (N * (N - 1)))[-N * idx :]
                            + ([-1] * N + [0] * (N * (N - 1)))[: -N * idx]
                            for idx in range(N)
                        ]
                    ),
                    np.zeros(shape=(N, N)),
                    np.zeros(shape=(N, N)),
                ),
                axis=1,
            ),
            np.concatenate(
                (
                    np.zeros(shape=(N, N)),
                    np.array(
                        [
                            np.array(
                                [
                                    ([1] + [0] * (N - 1))[-idx:]
                                    + ([1] + [0] * (N - 1))[:-idx]
                                ]
                                * N
                            ).flatten()
                            for idx in range(N)
                        ]
                    ),
                    np.eye(N),
                    np.zeros(shape=(N, N)),
                ),
                axis=1,
            ),
            np.array([[0] * N + [0] * (N * N) + [1] * N + [1] * N]),
            np.concatenate(
                (
                    -np.eye(N),
                    np.zeros(shape=(N, N * N)),
                    np.zeros(shape=(N, N)),
                    -np.eye(N),
                ),
                axis=1,
            ),
        ),
        axis=0,
    )

    # ξ(s) vector
    ξ_s = np.concatenate(
        (
            np.zeros(N),
            s,
            np.array([np.sum(s)]),
            np.zeros(N),
        ),
        axis=0,
    )

    return d_s, C_s, D_s, ξ_s


def s_gen_sgd(n=50):
    """Function that generates the scenario dependent
    second-stage variables by sampling demand

    """

    N, h, p, C, P, RETAILER_DEMANDS = get_transshipment_variables()

    for _ in range(n):

        # Get Second-stage variables
        s, p_s = random_demand(P=P, retailer_demands=RETAILER_DEMANDS)

        # Get scenario-dependent variables
        d_s, C_s, D_s, ξ_s = get_second_stage_variables(s)

        yield p_s, d_s, C_s, D_s, ξ_s


def s_gen_benders():
    """Function that generates the scenario dependent
    second-stage variables by enumerating all combinations of demand

    """

    N, h, p, C, P, RETAILER_DEMANDS = get_transshipment_variables()

    # Get Second-stage variables
    for s in itertools.product(*RETAILER_DEMANDS):

        p_s = np.prod(
            [
                P[np.argwhere(retailer_demand == D_i)[0][0]]
                for retailer_demand, D_i in zip(RETAILER_DEMANDS, s)
            ]
        )

        # Get scenario-dependent variables
        d_s, C_s, D_s, ξ_s = get_second_stage_variables(s)

        yield p_s, d_s, C_s, D_s, ξ_s

### 4.2. Stochastic Gradient Descent

In [6]:
sgd = StochasticGradientDescent(
    c=np.array([0.0] * 4),
    x0=np.array([0.0] * 4),
    first_stage_constraints=None,
    s_gen=s_gen_sgd,
    n=100,
    α=1e-01,
    ε=1e-01
)

sgd.solve()

==================== Iteration 0====================
Current Primal Objective Found: 15740.704 | Best Primal Objective Found: inf
==================== Iteration 50====================
Current Primal Objective Found: 2404.495 | Best Primal Objective Found: 2729.085
==================== Iteration 100====================
Current Primal Objective Found: 289.641 | Best Primal Objective Found: 264.472
==================== Iteration 150====================
Current Primal Objective Found: 262.191 | Best Primal Objective Found: 229.16
==================== Iteration 200====================
Current Primal Objective Found: 292.347 | Best Primal Objective Found: 197.497
==================== Iteration 250====================
Current Primal Objective Found: 290.276 | Best Primal Objective Found: 197.497
==================== Iteration 300====================
Current Primal Objective Found: 177.405 | Best Primal Objective Found: 167.574
==================== Iteration 350====================
Current Pri

(130.407359788351,
 array([100.66088181, 200.53971283, 150.72409382, 170.48564625]),
 499)

### 4.3. Benders

In [7]:
benders = Benders(
    c=np.array([0] * 4),
    first_stage_constraints=None,
    s_gen=s_gen_benders,
)

benders.solve()

==================== Iteration 0====================
Lower Bound: 0.0 | Upper Bound: 2480.0
==================== Iteration 1====================
Lower Bound: 0.0 | Upper Bound: 1041.897
==================== Iteration 2====================
Lower Bound: 0.0 | Upper Bound: 115.972
==================== Iteration 3====================
Lower Bound: 0.0 | Upper Bound: 115.972
==================== Iteration 4====================
Lower Bound: 0.0 | Upper Bound: 115.972
==================== Iteration 5====================
Lower Bound: 48.645 | Upper Bound: 115.972
==================== Iteration 6====================
Lower Bound: 49.022 | Upper Bound: 115.972
==================== Iteration 7====================
Lower Bound: 51.52 | Upper Bound: 115.972
==================== Iteration 8====================
Lower Bound: 66.869 | Upper Bound: 92.555
==================== Iteration 9====================
Lower Bound: 67.483 | Upper Bound: 92.555
==================== Iteration 10====================
Lowe

(80.52286246994309,
 array([108.50808924, 214.06507897, 159.7243887 , 187.70244296]),
 18)

### 4.4. Stochastic Gradient Descent Vs. Benders

In [ ]:
def run_experiment_sgd(n_scenario, α, ε):

    optimal_obj_val, optimal_order_up_to_quantity, k = StochasticGradientDescent(
        c=np.array([0.0] * 4),
        x0=np.array([0.0] * 4),
        first_stage_constraints=None,
        s_gen=s_gen_sgd,
        n=n_scenario,
        α=α,
        ε=ε,
        log=False,
    ).solve()

    return optimal_obj_val, optimal_order_up_to_quantity, k


def run_experiment_grid_sgd(
    n_experiments=100, n_scenarios=[50, 100, 1000], αs=[1e0, 1e-01, 1e-02], εs=[1e-03, 1e-04, 1e-05]
):

    results = []

    for n_scenario in n_scenarios:

        for α in αs:
            
            for ε in εs:

                (
                    optimal_obj_vals_sgd,
                    optimal_order_up_to_quantities_sgd,
                    n_iterations_sgd,
                    experiment_times,
                ) = ([], [], [], [])

                # Run Experiments
                for k in range(n_experiments):

                    if k % 10 == 0:
                        print(
                            f"Experimenting with n_scenario={n_scenario} | α={α} | ε={ε} | k={k}..."
                        )

                    start = time.time()
                    optimal_obj_val, optimal_order_up_to_quantity, n_iterations = run_experiment_sgd(
                        n_scenario, α, ε
                    )
                    end = time.time()

                    optimal_obj_vals_sgd.append(optimal_obj_val)
                    optimal_order_up_to_quantities_sgd.append(optimal_order_up_to_quantity)
                    n_iterations_sgd.append(n_iterations)
                    experiment_times.append(end - start)

                optimal_obj_vals_sgd = np.round(np.array(optimal_obj_vals_sgd), 2)
                optimal_order_up_to_quantities_sgd = np.round(np.array(
                    optimal_order_up_to_quantities_sgd
                ), 2)
                n_iterations_sgd = np.array(n_iterations_sgd)
                experiment_times = np.round(np.array(experiment_times), 2)

                results.append(
                    (
                        n_scenario,
                        α,
                        ε,
                        optimal_obj_vals_sgd.mean(axis=0),
                        optimal_obj_vals_sgd.std(axis=0),
                        optimal_obj_vals_sgd.max(axis=0),
                        optimal_obj_vals_sgd.min(axis=0),
                        optimal_order_up_to_quantities_sgd.mean(axis=0),
                        optimal_order_up_to_quantities_sgd.std(axis=0),
                        optimal_order_up_to_quantities_sgd.max(axis=0),
                        optimal_order_up_to_quantities_sgd.min(axis=0),
                        n_iterations_sgd.mean(axis=0),
                        n_iterations_sgd.std(axis=0),
                        n_iterations_sgd.max(axis=0),
                        n_iterations_sgd.min(axis=0),
                        experiment_times.mean(axis=0),
                        experiment_times.std(axis=0),
                        experiment_times.max(axis=0),
                        experiment_times.min(axis=0),
                    )
                )

                pd.DataFrame(
                    results,
                    columns=[
                        "Number of Scenarios",
                        "Learning rate α",
                        "Tolerance for Subgradient Early Stopping ε",
                        "Mean(Objective Value)",
                        "Std(Objective Value)",
                        "Max(Objective Value)",
                        "Min(Objective Value)",
                        "Mean(Optimal Order-Up-To Quantitites)",
                        "Std(Optimal Order-Up-To Quantitites)",
                        "Max(Optimal Order-Up-To Quantitites)",
                        "Min(Optimal Order-Up-To Quantitites)",
                        "Mean(Number of Iterations)",
                        "Std(Number of Iterations)",
                        "Max(Number of Iterations)",
                        "Min(Number of Iterations)",
                        "Mean(Experiment Times)",
                        "Std(Experiment Times)",
                        "Max(Experiment Times)",
                        "Min(Experiment Times)",
                    ],
                ).to_csv("sgd_results.csv")

    return pd.DataFrame(
        results,
        columns=[
            "Number of Scenarios",
            "Learning rate α",
            "Tolerance for Subgradient Early Stopping ε",
            "Mean(Objective Value)",
            "Std(Objective Value)",
            "Max(Objective Value)",
            "Min(Objective Value)",
            "Mean(Optimal Order-Up-To Quantitites)",
            "Std(Optimal Order-Up-To Quantitites)",
            "Max(Optimal Order-Up-To Quantitites)",
            "Min(Optimal Order-Up-To Quantitites)",
            "Mean(Number of Iterations)",
            "Std(Number of Iterations)",
            "Max(Number of Iterations)",
            "Min(Number of Iterations)",
            "Mean(Experiment Times)",
            "Std(Experiment Times)",
            "Max(Experiment Times)",
            "Min(Experiment Times)",
        ],
    )


sgd_results = run_experiment_grid_sgd(
    n_experiments=100, n_scenarios=[10, 50, 100, 200], αs=[1e-01], εs=[1e-01]
)
sgd_results

Experimenting with n_scenario=10 | α=0.1 | ε=0.1 | k=0...
Early Stopping condition satisfied.
Early Stopping condition satisfied.
Early Stopping condition satisfied.
Early Stopping condition satisfied.
Early Stopping condition satisfied.
Early Stopping condition satisfied.
Early Stopping condition satisfied.
Early Stopping condition satisfied.
Early Stopping condition satisfied.
Early Stopping condition satisfied.
Experimenting with n_scenario=10 | α=0.1 | ε=0.1 | k=10...
Early Stopping condition satisfied.
Early Stopping condition satisfied.
Early Stopping condition satisfied.
Early Stopping condition satisfied.
Early Stopping condition satisfied.
Early Stopping condition satisfied.
Early Stopping condition satisfied.
Early Stopping condition satisfied.
Early Stopping condition satisfied.
Early Stopping condition satisfied.
Experimenting with n_scenario=10 | α=0.1 | ε=0.1 | k=20...
Early Stopping condition satisfied.
Early Stopping condition satisfied.
Early Stopping condition satisfi

In [9]:
pd.read_csv("./sgd_results.csv", index_col=[0])

,Number of Scenarios,Learning rate α,Tolerance for Subgradient Early Stopping ε,Mean(Objective Value),Std(Objective Value),Max(Objective Value),Min(Objective Value),Mean(Optimal Order-Up-To Quantitites),Std(Optimal Order-Up-To Quantitites),Max(Optimal Order-Up-To Quantitites),Min(Optimal Order-Up-To Quantitites),Mean(Number of Iterations),Std(Number of Iterations),Max(Number of Iterations),Min(Number of Iterations),Mean(Experiment Times),Std(Experiment Times),Max(Experiment Times),Min(Experiment Times)
0,10,0.1,0.1,16.5198,6.759651,55.55,8.53,[148.4059 157.5988 154.6814 157.4469],[1.96487307 2.73314115 2.02832641 2.73398928],[150.08 162.47 156.3 162.31],[138.58 144.07 143.93 143.92],684.31,78.390267,980,560,28.8340,3.309319,41.45,23.71
1,50,0.1,0.1,53.6970,4.858717,64.30,43.98,[100.6798 200.597 150.6084 170.606 ],[0.37629504 0.35315011 0.34831227 0.34898424],[101.65 201.48 151.52 171.53],[100.02 200.01 150.02 170.03],786.58,40.985163,915,729,166.0559,8.613365,193.08,153.22
2,100,0.1,0.1,127.4597,9.509603,152.36,103.51,[101.2032 201.0891 151.1315 171.1326],[0.74875481 0.63293459 0.63881668 0.62607926],[104.29 202.48 152.68 172.56],[100.04 200.01 150.03 170.04],598.64,205.785933,1469,360,253.2370,87.046833,620.72,152.10


In [7]:
def run_experiment_grid_benders(n_experiments=100):

    results = []

    (
        optimal_obj_vals_benders,
        optimal_order_up_to_quantities_benders,
        n_iterations_benders,
        experiment_times,
    ) = ([], [], [], [])

    # Run Experiments
    for k in range(n_experiments):

        if k % 10 == 0:
            print(f"Experimenting with k={k}...")

        start = time.time()
        optimal_obj_val, optimal_order_up_to_quantity, n_iterations = Benders(
            c=np.array([0] * 4),
            first_stage_constraints=None,
            s_gen=s_gen_benders,
            log=False,
        ).solve()
        end = time.time()

        optimal_obj_vals_benders.append(optimal_obj_val)
        optimal_order_up_to_quantities_benders.append(optimal_order_up_to_quantity)
        n_iterations_benders.append(n_iterations)
        experiment_times.append(end - start)

    optimal_obj_vals_benders = np.round(np.array(optimal_obj_vals_benders), 2)
    optimal_order_up_to_quantities_benders = np.round(np.array(
        optimal_order_up_to_quantities_benders
    ), 2)
    n_iterations_benders = np.array(n_iterations_benders)
    experiment_times = np.round(np.array(experiment_times), 2)

    results.append(
        (
            5 ** 4,
            1,
            optimal_obj_vals_benders.mean(axis=0),
            optimal_obj_vals_benders.std(axis=0),
            optimal_obj_vals_benders.max(axis=0),
            optimal_obj_vals_benders.min(axis=0),
            optimal_order_up_to_quantities_benders.mean(axis=0),
            optimal_order_up_to_quantities_benders.std(axis=0),
            optimal_order_up_to_quantities_benders.max(axis=0),
            optimal_order_up_to_quantities_benders.min(axis=0),
            n_iterations_benders.mean(axis=0),
            n_iterations_benders.std(axis=0),
            n_iterations_benders.max(axis=0),
            n_iterations_benders.min(axis=0),
            experiment_times.mean(axis=0),
            experiment_times.std(axis=0),
            experiment_times.max(axis=0),
            experiment_times.min(axis=0),
        )
    )

    return pd.DataFrame(
        results,
        columns=[
            "Number of Scenarios",
            "Number of Iterations",
            "Mean(Objective Value)",
            "Std(Objective Value)",
            "Max(Objective Value)",
            "Min(Objective Value)",
            "Mean(Optimal Order-Up-To Quantitites)",
            "Std(Optimal Order-Up-To Quantitites)",
            "Max(Optimal Order-Up-To Quantitites)",
            "Min(Optimal Order-Up-To Quantitites)",
            "Mean(Number of Iterations)",
            "Std(Number of Iterations)",
            "Max(Number of Iterations)",
            "Min(Number of Iterations)",
            "Mean(Experiment Times)",
            "Std(Experiment Times)",
            "Max(Experiment Times)",
            "Min(Experiment Times)",
        ],
    )


benders_results = run_experiment_grid_benders(n_experiments=100)
benders_results.to_csv("benders_results.csv")
benders_results

Experimenting with k=0...
Experimenting with k=10...
Experimenting with k=20...
Experimenting with k=30...
Experimenting with k=40...
Experimenting with k=50...
Experimenting with k=60...
Experimenting with k=70...
Experimenting with k=80...
Experimenting with k=90...


,Number of Scenarios,Number of Iterations,Mean(Objective Value),Std(Objective Value),Max(Objective Value),Min(Objective Value),Mean(Optimal Order-Up-To Quantitites),Std(Optimal Order-Up-To Quantitites),Max(Optimal Order-Up-To Quantitites),Min(Optimal Order-Up-To Quantitites),Mean(Number of Iterations),Std(Number of Iterations),Max(Number of Iterations),Min(Number of Iterations),Mean(Experiment Times),Std(Experiment Times),Max(Experiment Times),Min(Experiment Times)
0,625,1,80.52,1.421085e-14,80.52,80.52,"[108.51000000000016, 214.0699999999998, 159.71...","[1.5631940186722204e-13, 1.9895196601282805e-1...","[108.51, 214.07, 159.72, 187.7]","[108.51, 214.07, 159.72, 187.7]",18.0,0.0,18,18,38.7504,0.110408,39.2,38.53


#### 4.4.1 Comparing Mean(Optimal Order-Up-To Quantities)

In [ ]:
pd.options.display.max_colwidth = 100

In [12]:
import pandas as pd

In [14]:
pd.read_csv("./sgd_results.csv", index_col=[0])

,Number of Scenarios,Learning rate α,Tolerance for Subgradient Early Stopping ε,Mean(Objective Value),Std(Objective Value),Max(Objective Value),Min(Objective Value),Mean(Optimal Order-Up-To Quantitites),Std(Optimal Order-Up-To Quantitites),Max(Optimal Order-Up-To Quantitites),Min(Optimal Order-Up-To Quantitites),Mean(Number of Iterations),Std(Number of Iterations),Max(Number of Iterations),Min(Number of Iterations),Mean(Experiment Times),Std(Experiment Times),Max(Experiment Times),Min(Experiment Times)
0,10,0.1,0.1,16.5198,6.759651,55.55,8.53,[148.4059 157.5988 154.6814 157.4469],[1.96487307 2.73314115 2.02832641 2.73398928],[150.08 162.47 156.3 162.31],[138.58 144.07 143.93 143.92],684.31,78.390267,980,560,28.8340,3.309319,41.45,23.71
1,50,0.1,0.1,53.6970,4.858717,64.30,43.98,[100.6798 200.597 150.6084 170.606 ],[0.37629504 0.35315011 0.34831227 0.34898424],[101.65 201.48 151.52 171.53],[100.02 200.01 150.02 170.03],786.58,40.985163,915,729,166.0559,8.613365,193.08,153.22
2,100,0.1,0.1,127.4597,9.509603,152.36,103.51,[101.2032 201.0891 151.1315 171.1326],[0.74875481 0.63293459 0.63881668 0.62607926],[104.29 202.48 152.68 172.56],[100.04 200.01 150.03 170.04],598.64,205.785933,1469,360,253.2370,87.046833,620.72,152.10


In [15]:
pd.read_csv("./benders_results.csv", index_col=[0])

,Number of Scenarios,Number of Iterations,Mean(Objective Value),Std(Objective Value),Max(Objective Value),Min(Objective Value),Mean(Optimal Order-Up-To Quantitites),Std(Optimal Order-Up-To Quantitites),Max(Optimal Order-Up-To Quantitites),Min(Optimal Order-Up-To Quantitites),Mean(Number of Iterations),Std(Number of Iterations),Max(Number of Iterations),Min(Number of Iterations),Mean(Experiment Times),Std(Experiment Times),Max(Experiment Times),Min(Experiment Times)
0,625,1,80.52,1.421085e-14,80.52,80.52,[108.51 214.07 159.72 187.7 ],[1.56319402e-13 1.98951966e-13 2.84217094e-13 ...,[108.51 214.07 159.72 187.7 ],[108.51 214.07 159.72 187.7 ],18.0,0.0,18,18,38.7504,0.110408,39.2,38.53


#### 4.4.2 Conclusion

We observe that benders goes through fewer iterations, and overall has a more accurate result.